In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/Aggregate_all_2019_Q1_segments'

In [2]:
def recurisve_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)

In [3]:
segment_1_list_Q1_regular=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/*.csv")
print("segment_1_list_Q1_regular: "+str(len(segment_1_list_Q1_regular)))

segment_2_list_Experian_0225=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Experian/Experian_Control_Test/output/*.csv")
print("segment_2_list_Experian_0225: "+str(len(segment_2_list_Experian_0225)))

segment_3_list_Experian_0412=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Experian_Email_Openers_Group/output/*.csv")
segment_3_list_Experian_0412=[x for x in segment_3_list_Experian_0412 if "summary" not in x]
print("segment_3_list_Experian_0412: "+str(len(segment_3_list_Experian_0412)))

segment_3_list_Experian_0427=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Experian_Email_Openers_Group/output_2nd_upload/*.csv")
print("segment_3_list_Experian_0427: "+str(len(segment_3_list_Experian_0427))) # just check

segment_4_list_new_rewards=list(recurisve_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/"))
segment_4_list_new_rewards=[x for x in segment_4_list_new_rewards if ".csv" in x]
df_segment_4_list_new_rewards=pd.DataFrame({"file_path":segment_4_list_new_rewards})
df_segment_4_list_new_rewards['date']=df_segment_4_list_new_rewards['file_path'].apply(lambda x: x.split("onthly_update/output_")[1][:10])
df_segment_4_list_new_rewards=df_segment_4_list_new_rewards[df_segment_4_list_new_rewards['date']>="2019-01-26"] # next run 02-23, already in Q1
df_segment_4_list_new_rewards=df_segment_4_list_new_rewards[df_segment_4_list_new_rewards['date']<"2019-05-04"] # uploaded 0504, so no target in Q1
segment_4_list_new_rewards=df_segment_4_list_new_rewards['file_path'].tolist()
print("segment_4_list_new_rewards: "+str(len(segment_4_list_new_rewards)))

dict_all_segments_files={"segment_1_list_Q1_regular":segment_1_list_Q1_regular,
                        "segment_2_list_Experian_0225":segment_2_list_Experian_0225,
                        "segment_3_list_Experian_0412":segment_3_list_Experian_0412,
                        "segment_4_list_new_rewards":segment_4_list_new_rewards}

segment_1_list_Q1_regular: 72
segment_2_list_Experian_0225: 10
segment_3_list_Experian_0412: 20
segment_3_list_Experian_0427: 20
segment_4_list_new_rewards: 15


In [4]:
df_all_aggregated_seg=pd.DataFrame()
df_file_summary=pd.DataFrame()

i_counter=0
for seg_wave in dict_all_segments_files.keys():
    seg_wave_name=seg_wave.split("_list_")[1]
    for file_path in dict_all_segments_files[seg_wave]:
        df=pd.read_csv(file_path,dtype=str)
        if df['segment'].nunique()!=1:
            print("multiple_segments_in_1_file",seg_wave,file_path, datetime.datetime.now())
        else:
            file_base_name=os.path.basename(file_path)
            df_all_aggregated_seg=df_all_aggregated_seg.append(df)
            
            segment_value=df['segment'].unique().tolist()[0]            
            count_ids=df['customer_id_hashed'].nunique()
            df_file_stat=pd.DataFrame({"file_base_name":file_base_name,"segment_value":segment_value,
                                      "id_counts":count_ids,"upload_wave":seg_wave_name},index=[i_counter])
            
            df_file_summary=df_file_summary.append(df_file_stat)
            i_counter+=1
            
            if i_counter%10==2:
                print(i_counter,datetime.datetime.now())
        
        

2 2019-06-11 15:08:26.573446
12 2019-06-11 15:09:36.426821
22 2019-06-11 15:09:49.369372
32 2019-06-11 15:13:29.110493
42 2019-06-11 15:22:15.600336
52 2019-06-11 15:39:36.001829
62 2019-06-11 15:54:12.408927
72 2019-06-11 15:54:33.381410
82 2019-06-11 16:04:29.094067
92 2019-06-11 16:09:25.235320
102 2019-06-11 16:12:22.278661
112 2019-06-11 16:18:30.613964


In [10]:
import pandas as pd
import numpy as np
import datetime
df_file_summary=pd.read_excel("./BL_2019Q1_LiveRamp_All_Segments_JL_2019-06-11.xlsx",dtype=str)
def def_test_control(x):
    if (("Experian_T" in x) | (x[:2]=="T_") | ("NewReward" in x) | (("T_" in x) & ("Experian" in x))):
        y="Test"
    elif (("_C_" in x) | (x[:2]=="C_")):
        y="Control"
    else:
        y=np.nan
    return y

df_file_summary['T_C_Group']=df_file_summary['file_base_name'].apply(lambda x: def_test_control(x))

In [11]:
writer=pd.ExcelWriter("./BL_2019Q1_LiveRamp_All_Segments_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_file_summary.to_excel(writer,"Q1_2019_segments_LR",index=False)
writer.save()

In [9]:
df_all_aggregated_seg.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/Aggregate_all_2019_Q1_segments/BL_2019Q1_all_segments_aggregated_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [10]:
df_all_aggregated_seg.head(2)

,Email_Address_hash,customer_id_hashed,customer_zip_code,email_address_hash,segment
0,NaN,24975afca413be4e03c4cece389765b9b479ceb476f8d1...,NaN,cf15cd4a38d13616ae31fb976e85c7759e655ac2cb3c81...,T_Quadrant I_P_L_2019Q1
1,NaN,3b45341b46de76883a6eab373af4ba6ca5ed390ba67ea2...,NaN,2403a3c80638d318f899685e6371989420789a526a631e...,T_Quadrant I_P_L_2019Q1


In [11]:
df_all_aggregated_seg.tail(4)

,Email_Address_hash,customer_id_hashed,customer_zip_code,email_address_hash,segment
14386,NaN,f229522fdc03e58373355f1766d69a2191259fc078c31b...,NaN,b6de85a13f6f26bd32bd7366fd270c44389eb4fc05fac3...,NewReward_042019_T
14387,NaN,e6a5518869788f68647f9c4c3a1bbab9b1d8acfb51ea42...,NaN,068ec8522dbb6b59534a94405ebaf8c582a47cf60dc312...,NewReward_042019_T
14388,NaN,155c89fb706c4bd909698f7a07954bdf6b45cdee8e4659...,94559,ca0e4cbe93735b728c3c05ec1229e1c63bf83587dcbd26...,NewReward_042019_T
14389,NaN,3d3f753057950d1fb0e0af8544cacf54c48f4edf2fa6d8...,NaN,7e8f2a6ea038871e886b4d267fe994357cdf98820a2ac8...,NewReward_042019_T
